In [1]:
from transformers import AutoModel, AutoTokenizer
import torch
import json
from tqdm import tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
model = AutoModel.from_pretrained(
    'mesolitica/llama2-embedding-2b-8k-contrastive',
    use_flash_attention_2 = True, 
    torch_dtype = torch.bfloat16,
    trust_remote_code = True,
)

config.json:   0%|          | 0.00/952 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/mesolitica/llama2-embedding-2b-8k-contrastive:
- modeling_contrastive.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/3.52G [00:00<?, ?B/s]

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [3]:
_ = model.cuda()

In [4]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/llama2-embedding-2b-8k-contrastive')

In [5]:
contexts, questions = [], []

with open('ultrachat-crossref-melayu-malay.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if l[1]['content_ms']:
            c = l[0]['content'].strip()
            q = l[1]['content_ms'].strip()
            if not len(c) or not len(q):
                continue
            contexts.append(c)
            questions.append(q)
            
len(contexts), len(questions)

(9673, 9673)

In [6]:
contexts_v = []
for i in tqdm(range(len(contexts))):
    input_ids = tokenizer([contexts[i]], 
        return_tensors = 'pt',
        padding = True
    )
    v = model.encode(input_ids.to('cuda')).detach().cpu().numpy()
    contexts_v.append(v)

100%|██████████| 9673/9673 [09:58<00:00, 16.17it/s] 


In [7]:
questions_v = []
for i in tqdm(range(len(contexts_v))):
    input_ids = tokenizer([questions[i]], 
        return_tensors = 'pt',
        padding = True
    )
    v = model.encode(input_ids.to('cuda')).detach().cpu().numpy()
    questions_v.append(v)

100%|██████████| 9673/9673 [01:01<00:00, 156.31it/s]


In [8]:
contexts_v_np = np.array(contexts_v)[:,0]
questions_v_np = np.array(questions_v)[:,0]

In [9]:
tops = {
    1: 0,
    3: 0,
    5: 0,
    10: 0,
}

for i in tqdm(range(len(questions_v_np))):
    argsort = np.argsort(cosine_similarity(questions_v_np[i].reshape(1, -1), contexts_v_np)[0])[::-1]
    for k in tops.keys():
        if i in argsort[:k]:
            tops[k] += 1

100%|██████████| 9673/9673 [04:25<00:00, 36.39it/s]


In [10]:
for k, v in tops.items():
    print(k, v / len(questions_v_np))

1 0.2671353251318102
3 0.49116096350666805
5 0.6091181639615424
10 0.739791171301561
